In [1]:
try:
    import inotify
except ImportError as e:
    !pip install inotify

In [2]:
from dask.distributed import Client
import subprocess as subp
import os
import dask
import inotify.adapters

In [3]:
from dask_cuda import LocalCUDACluster
cluster = LocalCUDACluster()

In [4]:
c = Client(cluster)
c

Client Scheduler: tcp://127.0.0.1:34054 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 50.39 GB


In [5]:
from nccl_example import NCCL_World
import random
from dask.distributed import wait

In [6]:
class Dask_MPI_Demo:
    
    def __init__(self, client):
        self.client = client
    
    @staticmethod
    def func_parse_host_port_(address):
        if '://' in address:
            address = address.rsplit('://', 1)[1]
        host, port = address.split(':')
        port = int(port)
        return host, port
    
    @staticmethod
    def func_init_(workerId, nWorkers, uniqueId):
        w = dask.distributed.get_worker()
        print("Hello World! from ip=%s worker=%s/%d" % \
              (w.address, w.name, nWorkers))
        a = NCCL_World(workerId, nWorkers)
        a.create_builder(uniqueId)

        return a
    
    @staticmethod
    def func_get_size_(world, r):
        return world.get_clique_size()
    
    @staticmethod
    def func_get_rank_(world, r):
        return world.get_rank()
    
    @staticmethod
    def func_get_device_(world, r):
        return world.get_device()
        
        
    def get_workers_(self):
        return list(map(lambda x: Dask_MPI_Demo.func_parse_host_port_(x), self.client.has_what().keys()))
    
    def init(self, uniqueId):
        workers = self.get_workers_()
        workers_indices = list(zip(workers, range(len(workers))))

        self.clients = [(idx, worker, self.client.submit(Dask_MPI_Demo.func_init_, 
                                           idx, 
                                           len(workers), 
                                           uniqueId,
                                           workers=[worker]))
             for worker, idx in workers_indices]
        
        print(str(self.clients))
        
    def get_clique_size(self):
        return [c.submit(Dask_MPI_Demo.func_get_size_, a, random.random()).result() for i, w, a in self.clients]
    

In [7]:
local_world = NCCL_World(-1, -1).unique_id()
local_world

b'nccl-50709-0'

In [8]:
demo = Dask_MPI_Demo(c)
demo.init(local_world)

[(0, ('127.0.0.1', 43536), <Future: status: pending, key: func_init_-a7e97a4a9cd45b41e46ef7d572f0f9d4>), (1, ('127.0.0.1', 46586), <Future: status: pending, key: func_init_-aec3e8330486f88a4880f6189399a073>)]


In [9]:
demo.clients

[(0,
  ('127.0.0.1', 43536),
  <Future: status: finished, type: NCCL_World, key: func_init_-a7e97a4a9cd45b41e46ef7d572f0f9d4>),
 (1,
  ('127.0.0.1', 46586),
  <Future: status: finished, type: NCCL_World, key: func_init_-aec3e8330486f88a4880f6189399a073>)]

In [10]:
rank_futures = demo.get_clique_size()

In [11]:
rank_futures

[2, 2]

In [12]:
print("Client Ranks: " + str(demo.get_client_ranks()))

Client Ranks: [1]


In [13]:
print("Server Rank: " + str(demo.get_server_rank()))

Server Rank: 0


In [17]:
demo.finalize()